In [1]:
import cv2
import moviepy.editor as mpy
from IPython.display import clear_output, display

In [2]:
class RangeSelect:
    playerName = 'PlayerWindow'
    
    def __init__(self, videoFileName):
        self.readVideo = cv2.VideoCapture(videoFileName) #要抽取视频的文件名
        self.fps, self.frameSum = self.GetInformation()  #25.026, 200.0
        
    def GetInformation(self):
        return self.readVideo.get(cv2.CAP_PROP_FPS), int(self.readVideo.get(cv2.CAP_PROP_FRAME_COUNT))
    
    @staticmethod
    def onTrackbar(pos, pRangeSelect):
        pRangeSelect.readVideo.set(cv2.CAP_PROP_POS_FRAMES, pos)
        posTime = int( pRangeSelect.readVideo.get(cv2.CAP_PROP_POS_MSEC) )
        
        status, frame = pRangeSelect.readVideo.read()
        frame = cv2.putText(frame, str(posTime)+'ms', (16, 32), 
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0))
        cv2.imshow(pRangeSelect.playerName, frame)
    
    def SelectRange(self):
        cv2.namedWindow(self.playerName)
        cv2.createTrackbar('FramePos', RangeSelect.playerName , 0, self.frameSum-1, 
                           lambda pos: RangeSelect.onTrackbar(pos, self))
        RangeSelect.onTrackbar(0, self)

        cv2.waitKey(0)   #To exit the window, DON’T CLICK THE RED CROSS, just tap any key on your keyboard!
        cv2.destroyAllWindows()

In [3]:
class SliceVideo:
    def __init__(self, videoFileName):
        self.videoFileName = videoFileName #要抽取视频的文件名
        self.fps = 20
    
    def ExtractVideo(self, outputFileName):
        cap = cv2.VideoCapture(self.videoFileName)
        pos0 = int( input('起始帧号:') )
        cap.set(cv2.CAP_PROP_POS_FRAMES, pos0)
            
        status,frame = cap.read()
        self.fps = cap.get(cv2.CAP_PROP_FPS)
        videoWriter = cv2.VideoWriter( outputFileName, 
                              cv2.VideoWriter_fourcc(*'mp4v'), 
                              self.fps, 
                              (frame.shape[1],frame.shape[0]), 
                              True)
       
        pos1 = int( input('终止帧号:') )
        while status and pos0 < pos1:
            clear_output(wait=True)
            print(f"正在处理第{pos0}帧")    # use display(f) if you encounter performance issues
            videoWriter.write(frame)   #新文件大约是原先3倍，two-pass encode?
            status,frame = cap.read()
            pos0 = pos0 + 1

        videoWriter.release()
        cap.release()
        
    def ExtractAudio(self, outputFileName):
        pos0 = float( input('起始秒数:') )
        pos1 = float( input('终止秒数:') )
        audioBackground = mpy.AudioFileClip(self.videoFileName).subclip(pos0, pos1)
        audioBackground.write_audiofile(outputFileName)
    
    @staticmethod
    def Small(inputFileName, outputFileName):
        big = cv2.VideoCapture(inputFileName)
        width0 = big.get(cv2.CAP_PROP_FRAME_WIDTH)
        height0 = big.get(cv2.CAP_PROP_FRAME_HEIGHT)
        scale = int( input(f'原始W={width0}, H={height0}. 输入缩小比例(2是原来的1/2):') )
        dimSmall = ( int(width0/scale), int(height0/scale) )
                             
        status,frame = big.read()
        fps = big.get(cv2.CAP_PROP_FPS)
        #input(f'{dimSmall, fps}')
        videoWriter = cv2.VideoWriter( outputFileName, 
                              cv2.VideoWriter_fourcc(*'mp4v'), 
                              fps, 
                              (dimSmall[0], dimSmall[1]), 
                              True)
        pos0 = 0
        while status:
            clear_output(wait=True)
            print(f"正在处理第{pos0}帧")
            frame2 = cv2.resize(frame, dimSmall)
            videoWriter.write(frame2)
            status,frame = big.read()
            pos0 = pos0 + 1

        videoWriter.release()
        big.release()
                      
    @staticmethod
    def VideoAddAudio(videoFileName, musicFileName, outputFileName, fps):
        origanVideo = mpy.VideoFileClip(videoFileName)
        audioBackground = mpy.AudioFileClip(musicFileName)
        finalVideo = origanVideo.set_audio(audioBackground)
        finalVideo.write_videofile(outputFileName, fps)

In [4]:
origalVideo = "video.mp4" #要抽取视频的文件名
fps = 25

task = '1'
while task != '0':
    task = input('0-exit, 1-选取范围(帧号和时间，只能按任意键退出)，2-存视频newSlice.mp4，3-存音频newSlice.mp3, 4-缩小视频, 5-合成newSlice2.mp4:')
    if task == '1':
        bigVideo = RangeSelect(origalVideo)
        bigVideo.SelectRange()
        fps = bigVideo.fps
        #del bigVideo
    elif task == '2':
        v = SliceVideo(origalVideo)
        outputFileName = input('outputFileName/mp4:')
        v.ExtractVideo(outputFileName)
    elif task == '3':
        v = SliceVideo(origalVideo)
        outputFileName = input('outputFileName/mp3:')
        v.ExtractAudio(outputFileName)
    elif task == '4':
        inputFileName  = input('inputFileName:')
        outputFileName = input('outputFileName:')
        SliceVideo.Small(inputFileName, outputFileName)
    elif task == '5':
        inputVideoFileName  = input('inputVideoFileName:')
        inputMusicFileName  = input('inputMusicFileName:')
        outputFileName      = input('outputFileName:')
        SliceVideo.VideoAddAudio(inputVideoFileName, inputMusicFileName, outputFileName, fps)
    else:
        print(task)

0-exit, 1-选取范围(帧号和时间，只能按任意键退出)，2-存视频newSlice.mp4，3-存音频newSlice.mp3, 4-缩小视频, 5-合成newSlice2.mp4:0
0
